# Implementing a Branch and Bound algorithm for Integer Problems, and applying it to finding the best Velogames teams

Velogames is a cycling fantasy game where, before each race you pick 9 riders for your team, and score points depending on how those riders did. Each rider has a pre-determined cost, the favorites costing more than the others. In total, you only have 100 credits to spend on your riders. A question that comes to mind is: given those 100 credits, what would be the best possible team to pick.

This translates nicely into an integer programming problem, which we implement and solve using the Branch and Bound Algorithm.

In [22]:
# imports, constants

import scipy.optimize as opt
import numpy as np
import queue
import pandas as pd

In [23]:
# Implementation of the Branch and Bound algorithm (probably quite inefficient)

class ILP():
    def __init__(self, c, A, b):
        self.c = c
        self.A = A
        self.b = b

    def relaxed_solve(self):
        c = self.c
        A = self.A
        b = self.b
        res = opt.linprog(c, A_ub=A, b_ub=b, bounds=bounds)
        return res

    def copy(self):
        return ILP(self.c, self.A, self.b)


def check_cut(I_cut):
    global z_best
    global x_best

    res = I_cut.relaxed_solve()
    if res.success:
        if res.fun >= z_best:
            return False
        else:
            if np.isclose(res.x, np.round(res.x), rtol=1e-9, atol=1e-9).all():
                z_best = res.fun
                x_best = res.x
                return False
            return True
    return False


def branch(I_branch):

    res = I_branch.relaxed_solve()
    dists = np.abs(res.x - np.round(res.x))
    idx = np.argmax(dists)
    split = np.floor(res.x[idx])

    I1 = I_branch.copy()
    I2 = I_branch.copy()

    new_row = np.zeros(I1.A.shape[1])
    new_row[idx] = 1
    I1.A = np.vstack((I1.A, new_row))
    I1.b = np.append(I1.b, split)

    new_row = np.zeros(I2.A.shape[1])
    new_row[idx] = - 1
    I2.A = np.vstack((I2.A, new_row))
    I2.b = np.append(I2.b, - split - 1)

    return I1, I2


def solve(I):
    Q = queue.LifoQueue()
    Q.put(I)
    while Q.qsize() > 0:
        I_temp = Q.get()
        if check_cut(I_temp):
            I1, I2 = branch(I_temp)
            Q.put(I1)
            Q.put(I2)
    return x_best, z_best

In [36]:
# Application of the Branch and Bound algorithm to the Branch and Bike problem
# Call the function with url being the "Riders" page for the Race you want to solve
# Set n_riders = 9 for stage races, 6 for classics

z_best = np.inf
x_best = None
bounds = (0,1)

def find_best_team(url, n_riders = 9):
    
    DF = pd.read_html(url)[0]
    
    c = - np.array(DF["Points"])
    A = np.vstack((DF["Cost"], np.ones(DF.shape[0]), -np.ones(DF.shape[0])))
    b = np.array([100, n_riders, -n_riders]) # constraint on cost, min number of riders, and max number of riders
    
    I = ILP(c, A, b)
    r, p = solve(I)
    
    riders = []
    for i in range(len(r)):
        if r[i]:
            riders.append(DF.iloc[i]["Rider"])
            
    print("Best team: ", *riders, sep = "\n")
    print( "with", -p, " points")
    print("Cost :", np.dot(r, DF["Cost"]))
    return riders, -p
    

In [37]:
%%time

riders, points = find_best_team('https://www.velogames.com/pn/2021/riders.php')

Best team: 
Primož Roglič
Maximilian Schachmann
Sam Bennett
Bryan Coquard
Christophe Laporte
Aurélien Paret-Peintre
Gino Mäder
Lucas Hamilton
Anthony Perez
with 4536.0  points
Cost : 100.0
CPU times: total: 469 ms
Wall time: 863 ms
